<a href="https://colab.research.google.com/github/Squirrelcoding/parking-meter/blob/main/Parking_meter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
import numpy as np
import pandas as pd

In [75]:
!pip3 install OSMPythonTools
!pip3 install gdal

In [76]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='poopnet-4fb22')

In [77]:
# TODO: Write some code that gets all of the boundaries of a parking lot given its ID or something
from OSMPythonTools.api import Api
api = Api()


In [78]:
import os
import geemap
from osgeo import gdal
from PIL import Image


def tif_to_png(filename: str | os.PathLike):
  pre = filename.split(".")[0]
  print(pre)
  tif = gdal.Open(filename)
  arr = tif.ReadAsArray().transpose(1, 2, 0)  # Convert CHW to HWC
  img = Image.fromarray(arr.astype(np.uint8))
  os.remove(filename)
  img.save(pre + '.png')

def download_parking_lot(way_id: int):
  way = api.query(f"way/{way_id}")
  roi = ee.Geometry.Polygon(way.geometry()['coordinates'])
  image_collection = (
      ee.ImageCollection('USDA/NAIP/DOQQ')
      .filterBounds(roi)  # Filter to images intersecting the ROI
      .filterDate('2022-01-01', '2023-12-31') # Filter to a recent time frame
      .sort('system:time_start', False) # Sort by date, newest first
  )

  mosaic = image_collection.mosaic()

  vis_params = {
      'bands': ['R', 'G', 'B'],
      'min': 0,
      'max': 255
  }

  clipped_image = mosaic.clip(roi).unmask()
  geemap.ee_export_image(clipped_image, filename=f'{way_id}.tif', scale=0.10394100844860077, region=roi)

In [79]:
id = 28452467
download_parking_lot(id)
tif_to_png(f"{id}.tif")

Generating URL ...
Please wait ...
Data downloaded to /content/28452467.tif
28452467


In [80]:
m = geemap.Map()

way = api.query(f"way/{id}")
roi = ee.Geometry.Polygon(way.geometry()['coordinates'])
image_collection = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterBounds(roi)  # Filter to images intersecting the ROI
    .filterDate('2022-01-01', '2023-12-31') # Filter to a recent time frame
    .sort('system:time_start', False) # Sort by date, newest first
)

mosaic = image_collection.mosaic()

vis_params = {
    'bands': ['R', 'G', 'B'],
    'min': 0,
    'max': 255
}
clipped_image = mosaic.clip(roi).unmask()
m.center_object(clipped_image)
m.add_layer(clipped_image, vis_params, "")

m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# Getting the most common color of the parking lot

In [92]:
from OSMPythonTools.overpass import Overpass

overpass = Overpass()

first_coordinate = (44.886320, -93.328578)  # (lat, lon)
second_coordinate = (44.88214, -93.31446)   # (lat, lon)

south = min(first_coordinate[0], second_coordinate[0])
north = max(first_coordinate[0], second_coordinate[0])
west = min(first_coordinate[1], second_coordinate[1])
east = max(first_coordinate[1], second_coordinate[1])

query = f"""
(
  way["amenity"="parking"]({south},{west},{north},{east});
  relation["amenity"="parking"]({south},{west},{north},{east});
);
out body;
>;
out skel qt;
"""

result = overpass.query(query)

In [93]:
def get_dominant_color(pil_img):
    img = pil_img.copy()
    img = img.convert("RGBA")
    img = img.resize((1, 1), resample=0)
    dominant_color = img.getpixel((0, 0))
    return dominant_color

In [95]:
n = 2
sum_colors = np.array([0.0, 0.0, 0.0])

colors = []
for parking_lot in result.toJSON()['elements'][0:n]:
  id = parking_lot['id']
  download_parking_lot(id)
  print(id)
  tif_to_png(f"{id}.tif")
  img = Image.open(f"{id}.png")
  color = np.array(get_dominant_color(img)[0:3]).astype(np.float32)
  sum_colors += color
  colors.append(color)
  os.remove(f"{id}.png")

sum_colors / n

Generating URL ...
Please wait ...
Data downloaded to /content/44707692.tif
44707692
44707692
Generating URL ...
Please wait ...
Data downloaded to /content/101843020.tif
101843020
101843020


array([200.5, 201. , 193.5])

In [96]:
colors

[array([186., 184., 173.], dtype=float32),
 array([215., 218., 214.], dtype=float32)]